## DFF and TFF (Toggle Flip-Flop)

In this example we create a toggle flip-flop (TFF) from a d-flip-flop (DFF). In `Magma`, finite state machines can be constructed by composing combinational logic with flop-flops register primitives.

In [1]:
import magma as m
from mantle import DFF

In [2]:
class TFF(m.Circuit):
    IO = ['O', m.Out(m.Bit)] + m.ClockInterface()
    @classmethod
    def definition(io):
        # instance a dff to hold the state of the toggle flip-flop - this needs to be done first
        dff = DFF()
        # compute the next state as the not of the old state ff.O
        io.O <= dff(~dff.O)
        #m.wiredefaultclock(io, dff)
        #m.wireclock(io, dff)
        
def tff():
    return TFF()()

Test using the python simulator.

In [3]:
from magma.simulator import PythonSimulator

sim = PythonSimulator(TFF, TFF.CLK)

sim.evaluate()
val = sim.get_value(TFF.O)
assert val == False

for i in range(10):
    val = not val
    
    sim.advance() # toggle clock - now High
    assert val == sim.get_value(TFF.O)

    sim.advance() # toggle clock - now Low
    assert val == sim.get_value(TFF.O)

print("Success!")

Success!


### Generate Verilog

In [4]:
m.compile("build/TFF", TFF, output="verilog")
%cat build/TFF.v

module DFF_init0_has_ceFalse_has_resetFalse_has_async_resetFalse (input  I, output  O, input  CLK);
wire [0:0] reg_P_inst0_out;
coreir_reg #(.init(0)) reg_P_inst0 (.I({I}), .CLK(CLK), .O(reg_P_inst0_out));
assign O = reg_P_inst0_out[0];
endmodule

module TFF (output  O, input  CLK);
wire  DFF_init0_has_ceFalse_has_resetFalse_has_async_resetFalse_inst0_O;
wire  magma_Bit_not_inst0_out;
DFF_init0_has_ceFalse_has_resetFalse_has_async_resetFalse DFF_init0_has_ceFalse_has_resetFalse_has_async_resetFalse_inst0 (.I(magma_Bit_not_inst0_out), .O(DFF_init0_has_ceFalse_has_resetFalse_has_async_resetFalse_inst0_O), .CLK(CLK));
magma_Bit_not magma_Bit_not_inst0 (.I(DFF_init0_has_ceFalse_has_resetFalse_has_async_resetFalse_inst0_O), .O(magma_Bit_not_inst0_out));
assign O = DFF_init0_has_ceFalse_has_resetFalse_has_async_resetFalse_inst0_O;
endmodule



Generate verilog with `coreir` modules included.

In [5]:
m.compile("build/TFF", TFF, output="coreir-verilog")
%cat build/TFF.v

module coreir_reg #(
    parameter width = 1,
    parameter clk_posedge = 1,
    parameter init = 1
) (
    input clk,
    input [width-1:0] in,
    output [width-1:0] out
);
  reg [width-1:0] outReg=init;
  wire real_clk;
  assign real_clk = clk_posedge ? clk : ~clk;
  always @(posedge real_clk) begin
    outReg <= in;
  end
  assign out = outReg;
endmodule

module corebit_not (
    input in,
    output out
);
  assign out = ~in;
endmodule

module DFF_init0_has_ceFalse_has_resetFalse_has_async_resetFalse (
    input I,
    output O,
    input CLK
);
wire [0:0] reg_P_inst0_out;
coreir_reg #(
    .clk_posedge(1'b1),
    .init(1'h0),
    .width(1)
) reg_P_inst0 (
    .clk(CLK),
    .in(I),
    .out(reg_P_inst0_out)
);
assign O = reg_P_inst0_out[0];
endmodule

module TFF (
    output O,
    input CLK
);
wire DFF_init0_has_ceFalse_has_resetFalse_has_async_resetFalse_inst0_O;
wire magma_Bit_not_inst0_out;
DFF_init0_has_ceFalse_has_resetFalse_has_async_resetFalse DFF_init0_has_ceFalse_has_re

In [6]:
m.compile("build/TFF", TFF, output="coreir")
%cat build/TFF.json

{"top":"global.TFF",
"namespaces":{
  "global":{
    "modules":{
      "DFF_init0_has_ceFalse_has_resetFalse_has_async_resetFalse":{
        "type":["Record",[
          ["I","BitIn"],
          ["O","Bit"],
          ["CLK",["Named","coreir.clkIn"]]
        ]],
        "instances":{
          "reg_P_inst0":{
            "genref":"coreir.reg",
            "genargs":{"width":["Int",1]},
            "modargs":{"clk_posedge":["Bool",true], "init":[["BitVector",1],"1'h0"]}
          }
        },
        "connections":[
          ["self.CLK","reg_P_inst0.clk"],
          ["self.I","reg_P_inst0.in.0"],
          ["self.O","reg_P_inst0.out.0"]
        ]
      },
      "TFF":{
        "type":["Record",[
          ["O","Bit"],
          ["CLK",["Named","coreir.clkIn"]]
        ]],
        "instances":{
          "DFF_init0_has_ceFalse_has_resetFalse_has_async_resetFalse_inst0":{
            "modref":"global.DFF_init0_has_ceFalse_has_resetFalse_has_async_resetFalse"
          },
          "magma

In [7]:
!coreir -i build/TFF.json -p instancecount

/Users/leonardtruong/miniconda3/lib/python3.7/site-packages/pysmt/walkers/generic.py:43: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if len(nodetypes) == 1 and isinstance(nodetypes[0], collections.Iterable):
An instance count of all the primitives
DFF_init0_has_ceFalse_has_resetFalse_has_async_resetFalse | instances in current | instances in children | 
  coreir_reg__width1 | 1 | 0

TFF | instances in current | instances in children | 
  corebit_not | 1 | 0
  coreir_reg__width1 | 0 | 1

{"top":"global.TFF",
"namespaces":{
  "global":{
    "modules":{
      "DFF_init0_has_ceFalse_has_resetFalse_has_async_resetFalse":{
        "type":["Record",[
          ["I","BitIn"],
          ["O","Bit"],
          ["CLK",["Named","coreir.clkIn"]]
        ]],
        "instances":{
          "reg_P_inst0":{
            "genref":"coreir.reg",
            "genargs":{"width":["Int",1]},
            "

In [8]:
from magma.simulator import PythonSimulator

sim = PythonSimulator(TFF, TFF.CLK)
for i in range(5):
    sim.advance_cycle()
    print(f"TFF.O = {sim.get_value(TFF.O)}")

TFF.O = False
TFF.O = True
TFF.O = False
TFF.O = True
TFF.O = False


In [9]:
from magma.simulator.coreir_simulator import CoreIRSimulator

sim = PythonSimulator(TFF, TFF.CLK)
for i in range(5):
    sim.advance_cycle()
    print(f"TFF.O = {sim.get_value(TFF.O)}")

TFF.O = False
TFF.O = True
TFF.O = False
TFF.O = True
TFF.O = False


In [10]:
import fault
tester = fault.Tester(TFF, TFF.CLK)
for i in range(5):
    tester.step(2)
    tester.print("TFF.O=%d\n", TFF.O)
tester.compile_and_run("verilator", disp_type='realtime')

Running command: verilator -Wall -Wno-INCABSPATH -Wno-DECLFILENAME --cc TFF.v --exe TFF_driver.cpp --top-module TFF
Running command: verilator --version
<STDOUT>
Verilator 4.016 2019-06-16 rev UNKNOWN_REV
</STDOUT>
Running command: make -C obj_dir -j -f VTFF.mk VTFF
<STDOUT>
clang++  -I.  -MMD -I/usr/local/Cellar/verilator/4.016/share/verilator/include -I/usr/local/Cellar/verilator/4.016/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fbracket-depth=4096 -Qunused-arguments -Wno-parentheses-equality -Wno-sign-compare -Wno-uninitialized -Wno-unused-parameter -Wno-unused-variable -Wno-shadow       -c -o TFF_driver.o ../TFF_driver.cpp
/usr/bin/perl /usr/local/Cellar/verilator/4.016/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include VTFF.cpp > VTFF__ALLcls.cpp
/usr/bin/perl /usr/local/Cellar/verilator/4.016/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include VTFF__Syms.cpp > VTFF__ALLsup.cpp
clang++  -I.  -MMD 